In [6]:
import os
import findspark
findspark.init(os.environ['SPARK_HOME'])

from pyspark.sql import SparkSession

# Inicializando a sessão do Spark
spark = SparkSession.builder.appName("Exemplo").getOrCreate()

## Text Cleaning and NLP
---

- **Tabela de frequência:**
* Relatica
* **Absoluta**

In [7]:
import nltk
from nltk.corpus import stopwords


from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import explode, col

In [8]:
# Baixar a lista de stopwords em português do NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aview\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Obter a lista de stopwords em português
stopwords_portugues = set(stopwords.words('portuguese'))

In [10]:
# Total de stopwords em portugues: 
print(f"Total de stopwords em portugues: {len(stopwords_portugues)}")

Total de stopwords em portugues: 207


In [12]:
inputPath = "..\data\clientes.csv"

# Carregue os dados
data_cliente = spark \
        .read\
        .format("csv")\
        .option("inferSchema", "true")\
        .option("header", "true")\
        .option("encoding", "UTF-8")\
        .csv(inputPath) \
        
data = data_cliente.filter("NM_CIDADE IS NOT NULL")
data.show(truncate=False)

+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+
|EN_RUA                        |NR_CEP  |NM_CIDADE             |SG_UF|NM_BAIRRO          |DS_LATITUDE        |DS_LONGITUDE      |CPF           |NOME           |CD_CLIENTE|
+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+
|RUA EXISTENTE                 |09340060|MAUA                  |SP   |VILA ANA           |NULL               |NULL              |595.958.963-00|Beatriz Silva  |0         |
|AVENIDA DOS TRABALHADORES     |13338050|INDAIATUBA            |SP   |VILA CASTELO BRANCO|-23.0887183        |-47.198916        |695.935.479-00|Maria Ferreira |1         |
|R OTAVIO DE MORAIS            |05351080|S PAULO               |SP   |C S FRANCISCO      |NULL               |NULL              |223.379.779

In [13]:
# Tokenize os textos
tokenizer = Tokenizer(inputCol="NM_CIDADE", outputCol="palavras")
tokenized = tokenizer.transform(data)
tokenized.show(truncate=False)

+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+---------------------------+
|EN_RUA                        |NR_CEP  |NM_CIDADE             |SG_UF|NM_BAIRRO          |DS_LATITUDE        |DS_LONGITUDE      |CPF           |NOME           |CD_CLIENTE|palavras                   |
+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+---------------------------+
|RUA EXISTENTE                 |09340060|MAUA                  |SP   |VILA ANA           |NULL               |NULL              |595.958.963-00|Beatriz Silva  |0         |[maua]                     |
|AVENIDA DOS TRABALHADORES     |13338050|INDAIATUBA            |SP   |VILA CASTELO BRANCO|-23.0887183        |-47.198916        |695.935.479-00|Maria Ferreira |1         |[indaiatuba]               |


In [14]:
# Configurando o remover de stopwords
stopwords_remover = StopWordsRemover(inputCol="palavras", outputCol="TextoSemStopwords", stopWords=list(stopwords_portugues))

# Aplicando a remoção de stopwords (note que você deve ter a coluna "Tokens" do exemplo anterior)
df_sem_stopwords = stopwords_remover.transform(tokenized)

# Exibindo o DataFrame resultante
print("\nDataFrame Após Remoção de Stopwords:")
df_sem_stopwords.show(truncate=False)


DataFrame Após Remoção de Stopwords:
+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+---------------------------+-----------------------+
|EN_RUA                        |NR_CEP  |NM_CIDADE             |SG_UF|NM_BAIRRO          |DS_LATITUDE        |DS_LONGITUDE      |CPF           |NOME           |CD_CLIENTE|palavras                   |TextoSemStopwords      |
+------------------------------+--------+----------------------+-----+-------------------+-------------------+------------------+--------------+---------------+----------+---------------------------+-----------------------+
|RUA EXISTENTE                 |09340060|MAUA                  |SP   |VILA ANA           |NULL               |NULL              |595.958.963-00|Beatriz Silva  |0         |[maua]                     |[maua]                 |
|AVENIDA DOS TRABALHADORES     |13338050|INDAIATUBA            |SP

In [15]:
# Exploda as palavras em linhas
exploded = df_sem_stopwords.select(explode("TextoSemStopwords").alias("palavra"), "TextoSemStopwords", "NM_CIDADE")
exploded.groupBy(["palavra", "TextoSemStopwords", "NM_CIDADE"]).count().orderBy(col("count").desc()).show(truncate=False)

+-----------+----------------------+---------------------+------+
|palavra    |TextoSemStopwords     |NM_CIDADE            |count |
+-----------+----------------------+---------------------+------+
|paulo      |[sao, paulo]          |SAO PAULO            |726729|
|sao        |[sao, paulo]          |SAO PAULO            |726729|
|s          |[s, paulo]            |S PAULO              |116503|
|paulo      |[s, paulo]            |S PAULO              |116503|
|guarulhos  |[guarulhos]           |GUARULHOS            |91668 |
|campinas   |[campinas]            |CAMPINAS             |74731 |
|osasco     |[osasco]              |OSASCO               |52714 |
|campo      |[sao, bernardo, campo]|SAO BERNARDO DO CAMPO|51287 |
|sao        |[sao, bernardo, campo]|SAO BERNARDO DO CAMPO|51287 |
|bernardo   |[sao, bernardo, campo]|SAO BERNARDO DO CAMPO|51287 |
|sorocaba   |[sorocaba]            |SOROCABA             |47789 |
|campos     |[sao, jose, campos]   |SAO JOSE DOS CAMPOS  |46909 |
|jose     

In [16]:
# Conte a frequência das palavras
word_freq = exploded.groupBy("palavra").count().orderBy(col("count").desc())

# Mostre a tabela de frequência de palavras
word_freq.show()

+---------+------+
|  palavra| count|
+---------+------+
|      sao|938014|
|    paulo|854853|
|        s|153734|
|guarulhos| 93310|
|     jose| 88631|
| campinas| 75825|
|    preto| 70883|
|    campo| 65669|
|   campos| 59610|
| bernardo| 59541|
| ribeirao| 55053|
| paulista| 54155|
|   osasco| 53247|
|      rio| 51758|
|    santo| 50613|
|     mogi| 48504|
| sorocaba| 48330|
|    andre| 48117|
|    serra| 47390|
|   grande| 37711|
+---------+------+
only showing top 20 rows



In [17]:
word_freq.count()

5048

In [18]:
word_freq.toPandas().to_csv('word_freq.csv')